このサンプルは以下の公式サンプルをもとにしています。

- [amazon-sagemaker-examples/r_examples/r_in_sagemaker_processing/r_in_sagemaker_processing.ipynb](https://github.com/rabowskyb/amazon-sagemaker-examples/blob/r-in-processing/r_examples/r_in_sagemaker_processing/r_in_sagemaker_processing.ipynb)

# ファイルの説明

- preprocessing.R: Rで書かれた前処理のスクリプト
- Dockerfile: イメージ用のDOckerfile



# 1. 準備

boto3のインポートや各リソース名を定義します。

In [1]:
import boto3

account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

ecr_repository = 'r-in-sagemaker-processing'
tag = ':latest'

uri_suffix = 'amazonaws.com'
processing_repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, ecr_repository + tag)


SageMaker Studioにはdockerが未インストールでしたのでdockerをインストールします。ちなみにsudoも入ってないのでこちらもインストールします。

In [2]:
!apt install -y sudo
!sudo apt-get update
!sudo apt-get install -y \
    apt-transport-https \
    ca-certificates \
    curl \
    software-properties-common
!curl -fsSL https://download.docker.com/linux/debian/gpg | sudo apt-key add -
!sudo add-apt-repository \
   "deb [arch=amd64] https://download.docker.com/linux/debian \
   $(lsb_release -cs) \
   stable"
!sudo apt-get update
!sudo apt-get install -y docker-ce

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  sudo
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 1245 kB of archives.
After this operation, 3878 kB of additional disk space will be used.
Get:1 http://deb.debian.org/debian buster/main amd64 sudo amd64 1.8.27-1+deb10u2 [1245 kB]
Fetched 1245 kB in 0s (11.9 MB/s)3m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package sudo.
(Reading database ... 16492 files and directories currently installed.)
Preparing to unpack .../sudo_1.8.27-1+deb10u2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking sudo (1.8.27-1+deb10u2) ...
7Progress: [ 40%] [#######################...................................] 8Setting up sudo (

Processingジョブで実行するコンテナイメージを作成し、ECRへプッシュします。

In [12]:
!docker build -t $ecr_repository docker --no-cache
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)
!aws ecr create-repository --repository-name $ecr_repository
!docker tag {ecr_repository + tag} $processing_repository_uri
!docker push $processing_repository_uri

/bin/sh: 1: docker: not found
/bin/sh: 1: docker: not found

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'r-in-sagemaker-processing' already exists in the registry with id '119463424712'
/bin/sh: 1: docker: not found
/bin/sh: 1: docker: not found
